<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/PDSH-cover-small.png?raw=1">

*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Hierarchical Indexing](03.05-Hierarchical-Indexing.ipynb) | [Contents](Index.ipynb) | [Combining Datasets: Merge and Join](03.07-Merge-and-Join.ipynb) >

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.06-Concat-And-Append.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>


# 資料集的合併:Concat和Append

從不同資料來源中合併資料或將兩個不同的資料集串接在一起

In [2]:
import pandas as pd
import numpy as np

In [3]:
def make_df(cols, ind):
    """快速定義一個DataFrame作為範例使用"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


## 回想:在Numpy陣列中的串接



In [4]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [5]:
# 也可以改變串接的axis
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

## 使用``pd.concat``做簡單的串接

```python
# Signature in Pandas v0.18
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

``pd.concat()``可以用於``Series``或``DataFrame``物件的串接上，就像是``np.concatenate()``函式被用在單純的陣列串接一樣:

In [24]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
ser1

1    A
2    B
3    C
dtype: object

In [25]:
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

更高維度的物件像是``DataFrame``當然也可以使用:

In [17]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])

In [18]:
print(df1)

    A   B
1  A1  B1
2  A2  B2


In [19]:
print(pd.concat([df1, df2]))

    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


預設的情況下，在``DataFrame``中會被執行於列上(也就是``axis=0``)，然而就像是
``np.concatenate``一樣，``pd.concat``也可以讓我們透過設定axis以指定要被處理的軸。

In [27]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
df3

,A,B
0,A0,B0
1,A1,B1


In [30]:
print(pd.concat([df3, df4],axis=1))

    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1


### 重複的索引

``np.concatenate``和``pd.concat``最重要的差異在Pandas的串接會保留索引，即便會造成索引重複!

In [33]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # 使索引重複!
x

,A,B
0,A0,B0
1,A1,B1


In [34]:
print(pd.concat([x, y]))

    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


雖然可以在``DataFrame``中執行，但這樣的結果並不是我們想要的。
``pd.concat()``有幾種方法可解決:

#### 將此種重複情形當作錯誤

``verify_integrity``:簡單驗證`pd.concat()`產生的結果不要有重疊的索引。

當被設定為True時，若出現重複的索引會引起例外。

In [35]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


#### 忽略索引

若索引不重要，也可直接忽略。

``ignore_index``:直接忽略索引。當被設定為True時，串接後會直接在結果``Series``中的索引使用新的整數索引。

In [37]:
print(pd.concat([x, y], ignore_index=True))

    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


#### 加上多重索引鍵

另外的選擇是使用``keys``選項去替資料來源指定一個標籤，則結果會是一個包含資料的階層式索引序列

In [38]:
print(pd.concat([x, y], keys=['x', 'y']))

      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3


### 使用join進行串接



In [41]:
# 沒有資料的項目會以NA表示
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display(df5, df6, pd.concat([df5, df6]))

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


,B,C,D
3,B3,C3,D3
4,B4,C4,D4


,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


可透過``join``和``join_axes``的參數來處理。

預設是,join是輸入欄位的聯集(``join='outer'``)，但可以使用``join='inner'``改為交集

In [42]:
display(df5, df6,
        pd.concat([df5, df6], join='inner'))

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


,B,C,D
3,B3,C3,D3
4,B4,C4,D4


,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


``pd.concat``函數提供的功能，在打算合併兩個資料集時有非常多可以控制的行為。

### ``append()``方法

``Series``和``DataFrame``物件可以透過``append``方法，直接把陣列串在一起。比起呼叫``pd.concat([df1, df2])``, 簡單寫``df1.append(df2)``即可

In [49]:
display(df1, df2, df1.append(df2))

,A,B
1,A1,B1
2,A2,B2


,A,B
3,A3,B3
4,A4,B4


,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


不像Python list的``append()``和``extend()``,在Pandas的``append()``並不會改變原來的物件，而是建立一個合併資料後的新物件。


<!--NAVIGATION-->
< [Hierarchical Indexing](03.05-Hierarchical-Indexing.ipynb) | [Contents](Index.ipynb) | [Combining Datasets: Merge and Join](03.07-Merge-and-Join.ipynb) >

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.06-Concat-And-Append.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
